# Experimenting

In [83]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [84]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("game")

4

In [85]:
tools = [get_word_length]

In [86]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [87]:
llm_with_tools = llm.bind_tools(tools)

In [88]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [89]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [90]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LKjTzrN0dYZfndRRuH4A5lvL', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-77ae9b4c-761c-4769-860a-d9146b6320f5', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_LKjTzrN0dYZfndRRuH4A5lvL'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_LKjTzrN0dYZfndRRuH4A5lvL', 'index': 0}])], tool_call_id='call_LKjTzrN0dYZfndRRuH4A5lvL')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LKjTzrN0dYZfndRRuH4A5lvL', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'typ

# Current Soln

In [72]:
def select_best_option(text: str, options: list):
    prompt = f"Given the following text: '{text}', which of the following options is the best match?\n\nOptions:\n"
    for idx, option in enumerate(options, 1):
        prompt += f"{idx}. {option}\n"

    prompt += "\nPlease select the best option by number or name."

    response = llm.predict(prompt)

    selected_option = None
    for option in options:
        if option.lower() in response.lower():
            selected_option = option
            break

    return selected_option


The best option is: Restaurant with a great view


In [80]:
text = "i actually had a meeting come up on thursday"
options = ["Schedule New Appointment", "Reschedule an Existing Appointment", "Not Applicable"]

best_option = select_best_option(text, options)
print(f"text input: {text}")
print(f"This most likely aligns with |{best_option}|")

text input: i actually had a meeting come up on thursday
This most likely aligns with |Reschedule an Existing Appointment|


In [77]:
text_input = "who's better, Messi or Ronaldo?"
options_list = ["Schedule New Appointment", "Reschedule an Existing Appointment", "Not Applicable"]

best_option = select_best_option(text_input, options_list)
print(f"text input: {text_input}")
print(f"This most likely aligns with |{best_option}|")

text input: who's better, Messi or Ronaldo?
This most likely aligns with |Not Applicable|


In [81]:
text_input = "been having a fever lately"
options_list = ["Schedule New Appointment", "Reschedule an Existing Appointment", "Not Applicable"]

best_option = select_best_option(text_input, options_list)
print(f"text input: {text_input}")
print(f"This most likely aligns with |{best_option}|")

text input: been having a fever lately
This most likely aligns with |Schedule New Appointment|


Extensions: build into a langchain agent to use a tool to context check and make sure that prompts such as the follow as properly classified/

In [82]:
#TODO should classifiy as "Not Applicable"
text_input = "need a haircut soon"
options_list = ["Schedule New Appointment", "Reschedule an Existing Appointment", "Not Applicable"]

best_option = select_best_option(text_input, options_list)
print(f"text input: {text_input}")
print(f"This most likely aligns with |{best_option}|")

text input: need a haircut soon
This most likely aligns with |Schedule New Appointment|
